# Data cleaning

1) Read JSON data
2) covert timestamps to date time format
3) Drop unnecessary columns
4) Rename columns for ease of reference
5) Split time stamp column to date and time column
6) drop the original time stamp column
7) use glob module to clean multiple JSON files
8) combine all of htem into one df
9) safe to csv file 

# Columns to keep / drop 

* Keep 
    - Timestamp: track streaming habits over time
    - Ms_played: analyze how long I've listened to a song
    - Conn_country: listening while traveling
    - Track name: name of the song
    - Artist name
    - Shuffle
    - Skipped: see how many songs I've skipped

* Drop
    - Username
    - Platform
    - IP address
    - User agent
    - URI
    - Reason start, reason end
    - Offline
    - incognito mode

In [1]:
# #check current working directory
# import os
# print(os.getcwd())

# Basic data cleaning for ONE json file 

In [2]:
import pandas as pd

#load JSON file into df
df_1415 = pd.read_json(r'./Dataset/Spotify Extended/Streaming_History_Audio_2014-2015_0.json')

# convert ts column to datetime format
df_1415['ts'] = pd.to_datetime(df_1415['ts'])

# drop irrelevant columns
columns_to_drop = [
    'username', 'platform','ip_addr_decrypted', 'user_agent_decrypted', 'spotify_track_uri', 
    'episode_name','episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 
    'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode'
]

df_1415_cleaned = df_1415.drop(columns=columns_to_drop)

# rename columns
df_1415_cleaned = df_1415_cleaned.rename(columns={
    'ts': 'timestamp',
    'ms_played': 'duration',
    'conn_country': 'country',
    'master_metadata_track_name': 'song',
    'master_metadata_album_artist_name': 'artist',
    'master_metadata_album_album_name': 'album'
})

# convert timestamp to date and time columns
df_1415_cleaned['date'] = df_1415_cleaned['timestamp'].dt.date
df_1415_cleaned['time'] = df_1415_cleaned['timestamp'].dt.time

# drop the original time stamp column
df_1415_cleaned = df_1415_cleaned.drop(columns=['timestamp'])

df_1415_cleaned.head()

,duration,country,song,artist,album,date,time
0,29381,AU,Up and Away,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:33
1,13458,AU,Keep It Down Low,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:47
2,3436,AU,No Love In The Club,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:49
3,247520,AU,Am I Wrong,Nico & Vinz,Am I Wrong,2014-08-26,11:26:56
4,177352,AU,So Everything,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:29:54


# Cleaning data for MULTIPLE json files - GLOB 

- I have streaming history JSON file for the year:
    1) 2014 - 2015
    2) 2015 - 2017
    3) 2017 - 2022
    4) 2022 - 2023
    5) 2023 - 2024
    6) 2024

**Glob module**

- We don't want to waste time cleaning the date one by one, so we are going to use glob module to find all the pathnames matching a specified pattern
- This module is used for Unix-style pathname pattern expansion
- In our case, glob is used to get a list of all JSON files in a specific directory - as specified in the above.
- We want to apply the same prcoessing steps to multiple files without havign to manually specfy eahc file name 

**How does it work?**
- Import glob
- find files
- process the files
- Combine data frames

**Reusable Function for Data Cleaning** 
* Load JSON file into file_path
* Convert ts column to date time format
* drop irrelevant columns
* rename columns
* convert time stamp to date and time columns
* drop the original time stamp column
* load and clean all JSON files
* combine all cleaned dataframes into one

**Note: cross-refer with previous codes - they operate in a similar manner but this time, we do it for multiple JSON files**


In [3]:
import pandas as pd
import glob

def clean_streaming_data(file_path):
    df = pd.read_json(file_path)
    
    df['ts'] = pd.to_datetime(df['ts'])
    
    columns_to_drop = [
        'username', 'platform','ip_addr_decrypted', 'user_agent_decrypted', 'spotify_track_uri', 
        'episode_name','episode_show_name', 'spotify_episode_uri', 'reason_start', 'reason_end', 
        'shuffle', 'skipped', 'offline', 'offline_timestamp', 'incognito_mode'
    ]
    
    df_cleaned = df.drop(columns=columns_to_drop)
    
    df_cleaned = df_cleaned.rename(columns={
        'ts': 'timestamp',
        'ms_played': 'duration',
        'conn_country': 'country',
        'master_metadata_track_name': 'song',
        'master_metadata_album_artist_name': 'artist',
        'master_metadata_album_album_name': 'album'
    })
    
    df_cleaned['date'] = df_cleaned['timestamp'].dt.date
    df_cleaned['time'] = df_cleaned['timestamp'].dt.time
    
    df_cleaned = df_cleaned.drop(columns=['timestamp'])
    
    return df_cleaned

# load and clean all JSON files
file_paths = glob.glob('./Dataset/Spotify Extended/*.json')
dfs = [clean_streaming_data(file_path) for file_path in file_paths]

# combin all cleaned data frames into one
big_df = pd.concat(dfs, ignore_index=True)

# previw combined data frame

big_df.head()
    


,duration,country,song,artist,album,date,time
0,29381,AU,Up and Away,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:33
1,13458,AU,Keep It Down Low,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:47
2,3436,AU,No Love In The Club,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:22:49
3,247520,AU,Am I Wrong,Nico & Vinz,Am I Wrong,2014-08-26,11:26:56
4,177352,AU,So Everything,Envy,The Magic Soup And The Bittersweet Faces,2014-08-26,11:29:54


In [4]:
# # save combine DF to a csv file 
# csv_file_path = './Dataset/big_data.csv'
# big_df.to_csv(csv_file_path, index=False)